In [1]:
# Para trabajar con regex
# -----------------------------------------------------------------------
import re

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm



# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Librerias para generar gráficas
# ----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
!pip install openpyxl

### Cargamos los archivos xlsx de paro por municipios y leemos los datos que nos interesan

In [6]:
file_path_new = "datos/estadistica_paro_municipios.xlsx"
excel_data_new = pd.ExcelFile(file_path_new)
excel_data_new.sheet_names


['PORTADA',
 'Indice',
 'PARO A CORUÑA',
 'CONTRATOS CORUÑA',
 'PARO ALBACETE',
 'CONTRATOS ALBACETE',
 'PARO ALICANTE',
 'CONTRATOS ALICANTE',
 'PARO ALMERIA',
 'CONTRATOS ALMERIA',
 'PARO ALAVA',
 'CONTRATOS ARABA',
 'PARO ASTURIAS',
 'CONTRATOS ASTURIAS',
 'PARO AVILA',
 'CONTRATOS AVILA',
 'PARO BADAJOZ',
 'CONTRATOS BADAJOZ',
 'PARO BARCELONA',
 'CONTRATOS BARCELONA',
 'PARO BIZKAIA',
 'CONTRATOS BIZKAIA',
 'PARO BURGOS',
 'CONTRATOS BURGOS',
 'PARO CACERES',
 'CONTRATOS CACERES',
 'PARO CADIZ',
 'CONTRATOS CADIZ',
 'PARO CANTABRIA',
 'CONTRATOS CANTABRIA',
 'PARO CASTELLON',
 'CONTRATOS CASTELLON',
 'PARO CEUTA',
 'CONTRATOS CEUTA',
 'PARO CIUDAD REAL',
 'CONTRATOS CIUDAD REAL',
 'PARO CORDOBA',
 'CONTRATOS CORDOBA',
 'PARO CUENCA',
 'CONTRATOS CUENCA',
 'PARO GIPUZCOA',
 'CONTRATOS GIPUZCOA',
 'PARO GIRONA',
 'CONTRATOS GIRONA',
 'PARO GRANADA',
 'CONTRATOS GRANADA',
 'PARO GUADALAJARA',
 'CONTRATOS GUADALAJARA',
 'PARO HUELVA',
 'CONTRATOS HUELVA',
 'PARO HUESCA',
 'CONTRATOS H

In [7]:
# Cargar la hoja "PARO ALBACETE" en un DataFrame
df_paro_albacete = pd.read_excel(file_path_new, sheet_name="PARO ALBACETE")

# Mostrar las primeras filas del DataFrame para revisión
df_paro_albacete.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,"PARO REGISTRADO SEGÚN SEXO, EDAD Y SECTOR DE A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ALBACETE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DICIEMBRE 2022,NaN,NaN,NaN,NaN
3,NaN,MUNICIPIOS,TOTAL,SEXO Y EDAD,NaN,NaN,NaN,NaN,NaN,SECTORES,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,HOMBRES,NaN,NaN,MUJERES,NaN,NaN,AGRI- CULTURA,INDUS- TRIA,CONS- TRUCCIÓN,SERVICIOS,SIN EMPLEO ANTERIOR


In [8]:
# Limpiar el DataFrame eliminando filas vacías y reestableciendo los encabezados
df_paro_albacete_cleaned = pd.read_excel(file_path_new, sheet_name="PARO ALBACETE", header=4)

# Mostrar las primeras filas para verificar la estructura después de limpiar
df_paro_albacete_cleaned.head()


,Unnamed: 0,MUNICIPIOS,TOTAL,SEXO Y EDAD,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,SECTORES,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,HOMBRES,NaN,NaN,MUJERES,NaN,NaN,AGRI- CULTURA,INDUS- TRIA,CONS- TRUCCIÓN,SERVICIOS,SIN EMPLEO ANTERIOR
1,NaN,NaN,NaN,<25,25 - 44,>=45,<25,25 - 44,>=45,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001.0,ABENGIBRE,28,,<5,8,<5,5,10,<5,<5,<5,20,<5
4,2002.0,ALATOZ,39,<5,<5,11,<5,8,15,<5,<5,8,26,<5


In [9]:
# Unificar encabezados para simplificar la estructura
df_paro_albacete_cleaned.columns = [
    "Código", "Municipios", "Total", "Hombres <25", "Hombres 25-44", "Hombres >=45",
    "Mujeres <25", "Mujeres 25-44", "Mujeres >=45", "Agricultura", "Industria",
    "Construcción", "Servicios", "Sin Empleo Anterior"
]

# Eliminar filas con datos no relacionados (encabezados adicionales, NaN innecesarios)
df_paro_albacete_cleaned = df_paro_albacete_cleaned.dropna(subset=["Municipios"]).reset_index(drop=True)

df_paro_albacete_cleaned.head()


,Código,Municipios,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
0,2001.0,ABENGIBRE,28,,<5,8,<5,5,10,<5,<5,<5,20,<5
1,2002.0,ALATOZ,39,<5,<5,11,<5,8,15,<5,<5,8,26,<5
2,2003.0,ALBACETE,11185,458,1540,2173,439,2529,4046,517,1008,620,8028,1012
3,2004.0,ALBATANA,63,<5,<5,13,0,18,28,13,5,<5,39,<5
4,2005.0,ALBOREA,58,<5,5,21,<5,10,20,15,6,<5,31,<5


In [10]:
# Crear un diccionario para almacenar los DataFrames de todas las provincias
provinces_data = {}

# Identificar las hojas de interés relacionadas con "PARO"
paro_sheets = [sheet for sheet in excel_data_new.sheet_names if sheet.startswith("PARO")]

# Procesar cada hoja de "PARO" siguiendo los pasos realizados para Albacete
for sheet in paro_sheets:
    # Cargar la hoja correspondiente
    df_sheet = pd.read_excel(file_path_new, sheet_name=sheet, header=4)
    
    # Estandarizar los nombres de las columnas
    df_sheet.columns = [
        "Código", "Municipios", "Total", "Hombres <25", "Hombres 25-44", "Hombres >=45",
        "Mujeres <25", "Mujeres 25-44", "Mujeres >=45", "Agricultura", "Industria",
        "Construcción", "Servicios", "Sin Empleo Anterior"
    ]
    
    # Eliminar filas innecesarias y limpiar el DataFrame
    df_sheet = df_sheet.dropna(subset=["Municipios"]).reset_index(drop=True)
    
    # Guardar el DataFrame procesado en el diccionario
    provinces_data[sheet] = df_sheet

# Mostrar uno de los DataFrames para verificar el proceso
dataframe=provinces_data["PARO ALICANTE"]
dataframe.head()


,Código,Municipios,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
0,3001.0,ADSUBIA,24,<5,5,<5,0,5,10,<5,<5,,21,0
1,3002.0,AGOST,370,14,57,106,11,78,104,42,59,37,205,27
2,3003.0,AGRES,31,<5,5,13,0,5,6,<5,<5,<5,24,
3,3004.0,AIGÜES,70,<5,6,18,0,20,23,,<5,5,57,5
4,3005.0,ALBATERA,948,21,114,176,28,234,375,87,168,55,561,77


In [13]:
# Crear una lista para almacenar los DataFrames con la columna de provincia añadida
combined_data = []

# Iterar por cada provincia en el diccionario
for province, df in provinces_data.items():
    # Extraer el nombre de la provincia del nombre de la hoja
    province_name = province.replace("PARO ", "").strip()
    # Añadir una columna para la provincia
    df["Provincia"] = province_name
    # Añadir el DataFrame a la lista
    combined_data.append(df)

# Combinar todos los DataFrames en uno solo
df_combined = pd.concat(combined_data, ignore_index=True)

# Mostrar el DataFrame combinado
df_combined.head()


,Código,Municipios,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior,Provincia
0,15001.0,ABEGONDO,198,7,29,47,<5,37,74,8,26,12,139,13,A CORUÑA
1,15002.0,AMES,1487,26,234,338,35,369,485,22,102,88,1120,155,A CORUÑA
2,15003.0,ARANGA,55,<5,8,25,,15,6,<5,<5,<5,44,<5,A CORUÑA
3,15004.0,ARES,310,5,36,103,7,60,99,10,33,37,208,22,A CORUÑA
4,15005.0,ARTEIXO,1862,44,279,431,28,423,657,27,229,141,1311,154,A CORUÑA


In [17]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8186 entries, 0 to 8185
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Código               8134 non-null   float64
 1   Municipios           8186 non-null   object 
 2   Total                8186 non-null   object 
 3   Hombres <25          8186 non-null   object 
 4   Hombres 25-44        8186 non-null   object 
 5   Hombres >=45         8186 non-null   object 
 6   Mujeres <25          8186 non-null   object 
 7   Mujeres 25-44        8186 non-null   object 
 8   Mujeres >=45         8186 non-null   object 
 9   Agricultura          8186 non-null   object 
 10  Industria            8186 non-null   object 
 11  Construcción         8186 non-null   object 
 12  Servicios            8186 non-null   object 
 13  Sin Empleo Anterior  8186 non-null   object 
 14  Provincia            8186 non-null   object 
dtypes: float64(1), object(14)
memory usage

In [14]:
df_combined.to_csv("datos/paro_municipios_limpiados.csv", index=False)

In [15]:
df_paro_municipios = pd.read_csv("datos/paro_municipios_limpiados.csv")
df_paro_municipios.head()

,Código,Municipios,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior,Provincia
0,15001.0,ABEGONDO,198,7,29,47,<5,37,74,8,26,12,139,13,A CORUÑA
1,15002.0,AMES,1487,26,234,338,35,369,485,22,102,88,1120,155,A CORUÑA
2,15003.0,ARANGA,55,<5,8,25,,15,6,<5,<5,<5,44,<5,A CORUÑA
3,15004.0,ARES,310,5,36,103,7,60,99,10,33,37,208,22,A CORUÑA
4,15005.0,ARTEIXO,1862,44,279,431,28,423,657,27,229,141,1311,154,A CORUÑA


In [19]:
columns = df_paro_municipios.columns.tolist()
if 'Municipios' in columns and 'Provincia' in columns:
    columns.remove('Provincia')
    municipios_index = columns.index('Municipios')
    columns.insert(municipios_index + 1, 'Provincia')
    df_paro_municipios = df_paro_municipios[columns]

df_paro_municipios.head()

,Código,Municipios,Provincia,Total,Hombres <25,Hombres 25-44,Hombres >=45,Mujeres <25,Mujeres 25-44,Mujeres >=45,Agricultura,Industria,Construcción,Servicios,Sin Empleo Anterior
0,15001.0,ABEGONDO,A CORUÑA,198,7,29,47,<5,37,74,8,26,12,139,13
1,15002.0,AMES,A CORUÑA,1487,26,234,338,35,369,485,22,102,88,1120,155
2,15003.0,ARANGA,A CORUÑA,55,<5,8,25,,15,6,<5,<5,<5,44,<5
3,15004.0,ARES,A CORUÑA,310,5,36,103,7,60,99,10,33,37,208,22
4,15005.0,ARTEIXO,A CORUÑA,1862,44,279,431,28,423,657,27,229,141,1311,154
